In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from data_gen import *
from BP import *
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from DNN_BP import *

In [2]:
Nt, Nr, L =  8, 32, 6
batch = 5
Es = 10
device = torch.device('cuda:0')

Rx, Tx = 0.0, 0.0

s = torch.tensor([-3.0, -1.0, 1.0, 3.0],device=device)

In [38]:
# snr = torch.randint(low=0, high=6, size=(batch,Nr,1), device=device)*2.0+4.0
# noise_var = noise_variance(snr, Es, Nt)
# n = torch.empty(size=(batch,2*Nr,1), device=device)
# n[:,0::2,:] = n[:,1::2,:] = ((noise_var*0.5)**0.5)

# y, x, H = data_gen_16QAM_s(batch, Nt, Nr, n, Rx, Tx, device)

# y_s, x_s, H_s, n_s = shuffle_data(batch, y, x, H, n)

SNR = 14
noise = noise_variance(SNR, Es, Nt)
y, x, H = data_gen_16QAM(batch, Nt, Nr, noise, Rx, Tx, device)

In [39]:
damp = 0.6
w = 0.1
b = 0.1

In [40]:
P = torch.ones(size=(batch, 2*Nt, 4), device=device)*0.25
z, J, var_j = sMPD_transform(y, H, P, noise, Nr, device)

for l in range(8):
#     P = F.normalize(P, p=1, dim=2)
    s = torch.tensor([[-3.0, -1.0, 1.0, 3.0]], device=device).T
    J_diag = torch.unsqueeze(torch.diagonal(J, offset=0, dim1=1, dim2=2), dim=2)
    mean_v = torch.sum(J*torch.transpose(P@s,1,2),dim=2,keepdim=True) - J_diag*(P@s) 
    L = (2*(z-mean_v)-J_diag*(s+s[0]).T)*(J_diag*(s-s[0]).T)/(2.0*var_j)
    L1 = L - torch.max(L,dim=2,keepdim=True).values
    Post_P = w*torch.exp(L1) + b
    P = (1-damp)*Post_P + damp*P
    
x_p = s[torch.argmax(P, dim = 2)]

In [41]:
x_p.T == x.T

tensor([[[ True,  True,  True, False, False],
         [ True, False,  True, False, False],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True, False,  True],
         [ True,  True,  True, False,  True],
         [ True,  True,  True,  True, False],
         [ True,  True,  True,  True,  True],
         [ True,  True, False,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True, False, False,  True,  True],
         [False,  True,  True, False,  True],
         [ True,  True,  True, False,  True],
         [False,  True,  True, False,  True],
         [ True,  True, False,  True,  True],
         [ True,  True, False, False,  True]]], device='cuda:0')